### Importacion de Librerias

In [80]:
import simpy
import pandas as pd
import numpy as np
import random

### Definicion de clases, recursos y containers

In [81]:
class Piso:
    def __init__(self, env, id, num_parqueos):
        self.env = env
        self.id = id
        self.num_parqueos = simpy.Container(
            env,
            init=num_parqueos,
            capacity=num_parqueos
        )

In [82]:
class Estacionamiento:
    def __init__(self, env, parquimetros, pisos):
        self.env = env
        self.pisos = pisos
        self.parquimetros = simpy.Resource(env, capacity=parquimetros)

In [83]:
class Carro:
    def __init__(self,env, id, piso, estacionamiento):
        self.env = env
        self.id = id
        self.piso: Piso = piso
        self.estacionamiento: Estacionamiento = estacionamiento

        self.env.process(self.start())

    def start(self):

        piso_elegido = self.piso

        print(f"{self.env.now}: el carro con id {self.id} llego y quiere estacionarse en el piso {piso_elegido.id}")

        parquimetro = self.estacionamiento.parquimetros

        with parquimetro.request() as request:
            yield request
            print(f"{self.env.now}: El carro con id {self.id} esta entrando al estacionamiento")

            if piso_elegido.num_parqueos.level > 0:
                yield self.piso.num_parqueos.get(1)
                print(f"{self.env.now}: El carro {self.id} se estaciono en el piso que queria")
            else:
                encontrado = False
                for opcion_piso in self.estacionamiento.pisos:
                    if opcion_piso.num_parqueos.level > 0:
                        yield opcion_piso.num_parqueos.get(1)
                        print(f"{self.env.now}: El carro {self.id} no encontro espacio en el piso {piso_elegido.id} pero se estaciono en el piso {opcion_piso.id}")
                        encontrado = True
                        break

                if not encontrado:
                    print(f"{self.env.now}: El carro {self.id} se fue sin estacionarse. No habia espacio en ningun piso")


In [84]:
class Simulacion:
    def __init__(self, env, pisos_settings, num_parquimetros):
        self.env = env
        self.pisos = [Piso(env,p["id"], p["num_parqueos"]) for p in pisos_settings]
        self.estacionamiento = Estacionamiento(env, num_parquimetros, self.pisos)
        self.carros = []
        self.env.process(self.start())

    def start(self):
        while True:
            yield self.env.timeout(random.randint(1,4))

            for _ in range(random.randint(1,3)):
                carro = Carro(
                    self.env,
                    len(self.carros)+1,
                    random.choice(self.pisos),
                    self.estacionamiento
                )
                self.carros.append(carro)

### Inicio de Simulacion

In [85]:
env = simpy.Environment()

pisos_settings = [
    {
        "id":1,
        "num_parqueos":40
    },
    {
        "id":2,
        "num_parqueos":50
    }
    ]

num_parquimetros = 2

simulacion = Simulacion(env,pisos_settings,num_parquimetros)

env.run(until=120)

4: el carro con id 1 llego y quiere estacionarse en el piso 1
4: El carro con id 1 esta entrando al estacionamiento
4: El carro 1 se estaciono en el piso que queria
6: el carro con id 2 llego y quiere estacionarse en el piso 2
6: el carro con id 3 llego y quiere estacionarse en el piso 2
6: El carro con id 2 esta entrando al estacionamiento
6: El carro con id 3 esta entrando al estacionamiento
6: El carro 2 se estaciono en el piso que queria
6: El carro 3 se estaciono en el piso que queria
7: el carro con id 4 llego y quiere estacionarse en el piso 1
7: El carro con id 4 esta entrando al estacionamiento
7: El carro 4 se estaciono en el piso que queria
11: el carro con id 5 llego y quiere estacionarse en el piso 2
11: el carro con id 6 llego y quiere estacionarse en el piso 2
11: el carro con id 7 llego y quiere estacionarse en el piso 2
11: El carro con id 5 esta entrando al estacionamiento
11: El carro con id 6 esta entrando al estacionamiento
11: El carro 5 se estaciono en el piso qu

In [86]:
for s in simulacion.estacionamiento.pisos:
    print(f"{s.id}: {s.num_parqueos.level}")

1: 0
2: 0
